In [1]:
import requests
import pandas as pd
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

In [2]:
c_id = getpass.getpass()

········


In [3]:
c_secret = getpass.getpass()

········


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,client_secret=c_secret))

In [5]:
popvortex_100 = pd.read_csv('C:/Users/ebour/Documents/()_Ironhack DA Bootcamp/()_Labs/6_1_lab-web-scraping-single-page/popvortex_100.csv')

In [6]:
suggestion_database = pd.read_csv('clusterized_data.csv')

# MVP

In [7]:
import pickle
filename = 'spotify_cluster_model.sav'
cluster_model = pickle.load(open(filename, 'rb'))

In [8]:
filename_scaler = 'spotify_scaler.sav'
scaler = pickle.load(open(filename_scaler, 'rb'))

In [11]:
# song_input = input('Please fill in the name of a song you like : ')
song_input = 'poor lazarus'

In [12]:
song_input = song_input.lower()
top_100_check = popvortex_100[popvortex_100['title'] == song_input]
artist_input=''

len(top_100_check['title'])

0

In [21]:
if len(top_100_check['title']) > 0 :
    i = randint(1,100)
    while popvortex_100['title'][i] == song_input:   # used to avoid advising the same song as the input
        i = randint(1,100)
    proposed_title = popvortex_100['title'][i]
    proposed_artist = popvortex_100['artist'][i]
    output = 'Great choice ! This song is in the PopVortex Top 100. You should listen to "' + proposed_title + '" by ' + proposed_artist + '. You might like it !'
    print(output)

In [22]:
res = sp.search(q='track:' + song_input, type='track',limit = 50) 
total_songs = res['tracks']['total']
track_id = None

total_songs

45

In [23]:
if total_songs > 50:
    artist_input = input('Please fill in the name of a the artist : ')
    artist_input = artist_input.lower()
    res = sp.search(q='track:' + song_input + ' artist:' + artist_input, type='track',limit = 50)

res

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Apoor+lazarus&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6QmHysWvckkQR74oxmLmtz'},
       'href': 'https://api.spotify.com/v1/artists/6QmHysWvckkQR74oxmLmtz',
       'id': '6QmHysWvckkQR74oxmLmtz',
       'name': 'Dave Van Ronk',
       'type': 'artist',
       'uri': 'spotify:artist:6QmHysWvckkQR74oxmLmtz'}],
     'available_markets': ['AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BE',
      'BF',
      'BG',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
      'DO',
     

In [24]:
data = pd.DataFrame(res['tracks']['items'])
data['name'] = data['name'].apply(lambda x: x.lower())
data = data[data['name'] == song_input].reset_index(drop = True)
len(data['artists'])

15

In [ ]:
if len(data['artists']) == 0:
     output = 'We are sorry, but we cannot find any song with this name. Please check your spelling or try another song.'
else:
    if len(data['artists'])>1:
        if artist_input == '':
            artist_input = input('Please fill in the name of the artist : ')
            artist_input = artist_input.lower()
        artist_match = False
        for i in range(len(data['artists'])):
            art = data['artists'][i][0]['name'].lower()                    
            if art == artist_input:
                track_id = data['id'][i]
                artist_match=True
                break
        if artist_match == False:
            output = 'We are sorry, but we cannot find any song with this name and artist. Please check your spelling or try another song.'
            print(output)
    else:
        track_id =  data['id'][0]
    track_uri = 'spotify:track:' + track_id
    audio_features = pd.DataFrame(sp.audio_features(track_uri)[0], index=[0])[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    audio_feat_prep = scaler.transform(audio_features)
    song_input_cluster = cluster_model.predict(audio_feat_prep)[0]
    suggestion_list = suggestion_database[suggestion_database['cluster'] == song_input_cluster].reset_index(drop=True)
    j = randint(1,len(suggestion_list['track_name']))
    while suggestion_list['track_name'][j] == song_input:  #used to avoid advising the same song as the input (if ever in the suggestion list)
        j = randint(1,len(suggestion_list['track_name']))
    proposed_title = suggestion_list['track_name'][j]
    proposed_artist = suggestion_list['artist_name'][j]
    output = 'Your song is not in the PopVortex Top 100 but we still have a great advice for you : "' + proposed_title + '" by ' + proposed_artist + '. You might like it !'
print(output)

In [15]:
data

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AG, AL, AM, AO, AR, AT, AU, AZ, BA, BB, BE, B...",1,311240,False,{'isrc': 'USFI86201323'},{'spotify': 'https://open.spotify.com/track/45...,https://api.spotify.com/v1/tracks/45821oFb69KM...,45821oFb69KMrvsJfzTIRr,False,poor lazarus,21,None,6,track,spotify:track:45821oFb69KMrvsJfzTIRr
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,179200,False,{'isrc': 'GBWUL2187178'},{'spotify': 'https://open.spotify.com/track/3E...,https://api.spotify.com/v1/tracks/3EWlWpRGYG8N...,3EWlWpRGYG8N5fT0VDrkGb,False,poor lazarus,7,https://p.scdn.co/mp3-preview/75ee90d829f05691...,1,track,spotify:track:3EWlWpRGYG8N5fT0VDrkGb
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",1,308200,False,{'isrc': 'USFI86201323'},{'spotify': 'https://open.spotify.com/track/0T...,https://api.spotify.com/v1/tracks/0TX5cF6FreVM...,0TX5cF6FreVMt5Jate8aUO,False,poor lazarus,7,None,13,track,spotify:track:0TX5cF6FreVMt5Jate8aUO
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,243000,False,{'isrc': 'USVG20186041'},{'spotify': 'https://open.spotify.com/track/16...,https://api.spotify.com/v1/tracks/16qhdhLxcRPr...,16qhdhLxcRPrtWlRhiQnmU,False,poor lazarus,7,None,4,track,spotify:track:16qhdhLxcRPrtWlRhiQnmU
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AT, AU, BE, BS, CA, CH, CZ, DE, DK, ES, FI, F...",1,211160,False,{'isrc': 'USSH59186702'},{'spotify': 'https://open.spotify.com/track/6x...,https://api.spotify.com/v1/tracks/6x0V6AlGPIb5...,6x0V6AlGPIb5zSyUf2y2lO,False,poor lazarus,2,None,7,track,spotify:track:6x0V6AlGPIb5zSyUf2y2lO
5,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,201040,False,{'isrc': 'usl4q1876294'},{'spotify': 'https://open.spotify.com/track/4p...,https://api.spotify.com/v1/tracks/4ppK1YN2k9FM...,4ppK1YN2k9FMw95iSBoFbB,False,poor lazarus,0,https://p.scdn.co/mp3-preview/070106460103c0fc...,7,track,spotify:track:4ppK1YN2k9FMw95iSBoFbB
6,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,245466,False,{'isrc': 'USBT21605815'},{'spotify': 'https://open.spotify.com/track/0p...,https://api.spotify.com/v1/tracks/0pTsmKPynVSG...,0pTsmKPynVSGh3Q27HuTH1,False,poor lazarus,0,https://p.scdn.co/mp3-preview/f46c3e371ae0c160...,14,track,spotify:track:0pTsmKPynVSGh3Q27HuTH1
7,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,214466,False,{'isrc': 'ushm90456111'},{'spotify': 'https://open.spotify.com/track/5Z...,https://api.spotify.com/v1/tracks/5ZVGDqpJatYi...,5ZVGDqpJatYi25y4HdXQEh,False,poor lazarus,0,https://p.scdn.co/mp3-preview/da5b659efff9434c...,4,track,spotify:track:5ZVGDqpJatYi25y4HdXQEh
8,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,197693,False,{'isrc': 'USRO20182903'},{'spotify': 'https://open.spotify.com/track/1I...,https://api.spotify.com/v1/tracks/1IrsHJnlPEP2...,1IrsHJnlPEP2Hyn9f5Rke9,False,poor lazarus,1,None,3,track,spotify:track:1IrsHJnlPEP2Hyn9f5Rke9
9,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,205583,False,{'isrc': 'QZES618

In [17]:
data['artists']

0     [{'external_urls': {'spotify': 'https://open.s...
1     [{'external_urls': {'spotify': 'https://open.s...
2     [{'external_urls': {'spotify': 'https://open.s...
3     [{'external_urls': {'spotify': 'https://open.s...
4     [{'external_urls': {'spotify': 'https://open.s...
5     [{'external_urls': {'spotify': 'https://open.s...
6     [{'external_urls': {'spotify': 'https://open.s...
7     [{'external_urls': {'spotify': 'https://open.s...
8     [{'external_urls': {'spotify': 'https://open.s...
9     [{'external_urls': {'spotify': 'https://open.s...
10    [{'external_urls': {'spotify': 'https://open.s...
11    [{'external_urls': {'spotify': 'https://open.s...
12    [{'external_urls': {'spotify': 'https://open.s...
13    [{'external_urls': {'spotify': 'https://open.s...
14    [{'external_urls': {'spotify': 'https://open.s...
Name: artists, dtype: object

In [18]:
artist_input

'bob dylan'

In [19]:
for i in range(len(data['artists'])):
    art = data['artists'][i][0]['name'].lower() 
    print(art)

dave van ronk
rocks & salt
dave van ronk
ian & sylvia
mike cross
jeff hahn
the golden gate quartet
emily singleton
vera hall
peter pads
james bourke
dave plaehn
richard limbert
sister strange
julian fauth


In [27]:
sp.search(q='track: "poor lazarus"', type='track',limit = 50) 

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A+%22poor+lazarus%22&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6QmHysWvckkQR74oxmLmtz'},
       'href': 'https://api.spotify.com/v1/artists/6QmHysWvckkQR74oxmLmtz',
       'id': '6QmHysWvckkQR74oxmLmtz',
       'name': 'Dave Van Ronk',
       'type': 'artist',
       'uri': 'spotify:artist:6QmHysWvckkQR74oxmLmtz'}],
     'available_markets': ['AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BE',
      'BF',
      'BG',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
      'DO'

In [ ]:
song_advice()

In [ ]:
song_advice()

In [ ]:
song_advice()

In [ ]:
song_advice()